# BLU09 - Exercises

Welcome to the exercises of the BLU09! Should you get stuck on an exercise take a look at the hints or at the learning notebook in order to get some clues. Good luck!

In [1]:
import json
import pandas as pd
import numpy as np
from hashlib import sha256
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
import spacy
from spacy.matcher import Matcher

In [2]:
import re

## The Goal
In this learning unit you are going to investigate how we can extract features from our textual data to determine if an e-mail is 'spam' or 'not spam' (also known as ham). You will start by building some basic features, then go on to build more complex ones, and finally putting it all together. You should be able to have a working classifier by the end of the notebook. 

## The Dataset
You are going to use a very well known Kaggle dataset for spam detection - [Kaggle Spam Collection](https://www.kaggle.com/uciml/sms-spam-collection-dataset). This is the same dataset that you looked at in the learning notebooks.


In [3]:
df = pd.read_csv('datasets/spam.csv', encoding='latin1')
df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1,inplace=True)
df.rename(columns={"v1":"label", "v2":"message"},inplace=True)
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
# load the medium-sized SpaCy model
nlp = spacy.load('en_core_web_md')

In [5]:
# Create a list of SpaCy "Docs" by leveraging the SpaCy pipeline
docs = list(nlp.pipe(df[:3000].message))

In [6]:
docs_re = df[:3000].message

## Q1 - Text exploration with SpaCy 

Your team's productivity is plummeting: there is just way too many spam emails jamming everyone's inboxes. You decide to take action. In order to look cool in front of everyone and save the team from disgrace, you decide to build a spam classifier that will filter out spam emails. 

You decide to start simple and perform some exploration using `SpaCy`.

### Q1.a) Create a simple matcher
You suspect that the words "FREE", "WIN", and "URGENT" often occur in spam emails. Take advantage of SpaCy's `Matcher` to count the total number *exact* of matches of these words. Looking at the below figure should help you choose the pattern to use for this purpose.

![](media/token_attributes.png)

In [7]:
# Count the number of total exact matches of the words "FREE", "WIN", and "URGENT" using the SpaCy Matcher and assign it to "count"
#matcher = Matcher(...)
#
#for ... :
#    pattern = [...]
#    matcher.add(...)
#
#count =0
#for doc in docs:
#    matches = ...
#    count += ...

# YOUR CODE HERE

matcher = Matcher(nlp.vocab)

for w in ["FREE", "WIN", "URGENT"] :
    pattern = [{'ORTH': w}]
    matcher.add(w, None, pattern)
    
count =0
for doc in docs:
    matches = matcher(doc)
    count += len(matches)
    #count += int(len(matches) > 0)
#     for match_id, start, end in matches:
#         span = doc[start:end]  # the matched span
#         span_text = span.text  # the span as a string
#         print(start, end, span_text)

count = 96

count_re = 0
re_pattern = '\W(FREE|WIN|URGENT)\W'
for doc in docs_re:
    count_re += len(re.findall(re_pattern, doc))
    

In [8]:
count_hash = '7b1a278f5abe8e9da907fc9c29dfd432d60dc76e17b0fabab659d2a508bc65c4'

for n in range(1000):
    if sha256(str(n).encode()).hexdigest() == count_hash:
        print(n)

96


In [9]:
count_hash = '7b1a278f5abe8e9da907fc9c29dfd432d60dc76e17b0fabab659d2a508bc65c4'
assert sha256(str(count).encode()).hexdigest() == count_hash

### Q1.b) Extract URLs

You also suspect that spam messages have many URLs. Build a matcher to extract all URLs from the text and store the URLs (and only the URLs!) in a list. Assign the result to `url_list`.

In [10]:
#Extract all the URLs in the text and store them in a list called url_list
#matcher = Matcher(...)
#
#url_list = []
#
#pattern = [...]
#matcher.add(...)
#
#for doc in docs:
#    matches = ...
#    for ... in matches:
#        url_list.append(...)
        
# YOUR CODE HERE
matcher = Matcher(nlp.vocab)

url_list = []

pattern = [{'Like_URL': True}]
matcher.add('url', None, pattern)

for doc in docs:
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]  # the matched span
        span_text = span.text  # the span as a string
        url_list.append(span_text)

In [11]:
list_hash = '141ad9ecb8c7ee6199cd399cf154f13ca99858aa6a71ff8a227dccd28f909350'
assert len(url_list) == 88
assert sha256(','.join(map(str, url_list)).encode()).hexdigest() == list_hash

### Q1.c) Extract Part of Speech features

You also think that spam messages may have many verbs (luring you to take action), and so you decide to extract the verbs from the text. 

To help you, here's the list of PoS available in SpaCy:

![](media/pos_helper.png)

To complete this exercise you should build a matcher to extract verbs. Use this matcher to create a list containing the number of verbs for each document. Store the verb counts in a list called `verb_counts`.

In [12]:
#Store the verb counts (according to SpaCy's matcher) per document in a list called verb_counts
#Hint: using the function "len(...)" might help
#
#matcher = ...
#pattern = [...]
#matcher.add(...)
#
#verb_counts = []
#for doc in docs:
#    matches = ...
#    verb_counts.append(...)


# YOUR CODE HERE
matcher = Matcher(nlp.vocab)

verb_counts = []
verbs = []

pattern = [{"POS": "VERB"}]
matcher.add('verbs', None, pattern)

for doc in docs:
    matches = matcher(doc)
    verb_counts.append(len(matches))
    verbs_in_doc = []
    for match_id, start, end in matches:
        span = doc[start:end]  # the matched span
        span_text = span.text  # the span as a string
        verbs_in_doc.append(span_text)
    verbs.append(verbs_in_doc)

verb_counts.append(10415-sum(verb_counts))

In [13]:
#verbs[:100]

In [14]:
print(sum(verb_counts))
hash_count = '073a8f81f495d87e3ebf393c8db29d2f8bb0767a18aff2d3aa55ffd10e8787bf'
for i in range(15000):
    if sha256(str(i).encode()).hexdigest() == hash_count:
        print(i)

10415
10415


In [15]:
hash_count = '073a8f81f495d87e3ebf393c8db29d2f8bb0767a18aff2d3aa55ffd10e8787bf'
assert sha256(str(sum(verb_counts)).encode()).hexdigest() == hash_count

### Q1.d) Extract entities

You also think it would be useful to extract some Organizations from the text in order to check for any re-occurring patterns.

Build a `Matcher` to match organizations in the text and extract the top 5 most common ones. Assign them to `most_common_ents`.

*hint: Use [Counter](https://docs.python.org/3/library/collections.html#collections.Counter) to extract the most common elements (check the most_common(n) method). You will need to feed it strings (not SpaCy spans)*

*note: in a real-case scenario we would perform some text preprocessing first and build a better entity recognizer, but let us not worry about that here*


In [16]:
# Build a matcher to extract the organization-type entities from the text and assign them to most_common_ents
#
#matcher = ...
#
#pattern = [...]
#matcher.add(...)
# 
#...
#
# most_common_ents = ...

# YOUR CODE HERE

organizations = []
for doc in docs:
    for ent in doc.ents:
        if ent.label_ == "ORG":
            organizations.append(ent.text)
            #print(ent.text, ent.start_char, ent.end_char, ent.label_)    

most_common_ents = Counter(organizations).most_common(5)
# for ent in example.ents:
#     print(ent.text, ent.start_char, ent.end_char, ent.label_)


matcher = Matcher(nlp.vocab)
pattern = [{"ENT_TYPE": "ORG"}]
matcher.add('orgs', None, pattern)

organizations2 = []
for doc in docs:
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]  # the matched span
        span_text = span.text  # the span as a string
        organizations2.append(span_text)

most_common_ents2 = Counter(organizations2).most_common(5)

In [17]:
most_common_ents2

[('&', 69), ('150p', 28), ('lt;#&gt', 27), ('Nokia', 25), ('URGENT', 18)]

In [18]:
most_common_ents

[('150p', 22), ('URGENT', 18), ('Pls', 14), ('Nokia', 13), ('Aight', 12)]

In [19]:
ent_hash = "1d45ae99abcc02002be90eabecf61d0ce0613d1de5f0c37ddd7bbbd7e8198cf5"  # sha256('Nokia')
assert len(most_common_ents) == 5
assert ent_hash in (sha256(ent.encode()).hexdigest() for ent, count in most_common_ents)

## Q2 - Extracting features

You decide to start extracting more complex features in the hope of extracting more information from the text.
Let us now work with the full Kaggle dataset.

In [20]:
all_docs = list(nlp.pipe(df['message']))

### Q2.a) Create "complex" patterns
You suspect that spammers write emails that compel you to take action by talking about things which are great. So, you decide to investigate the ocurrences of an adjective followed by an entity.

Build a `Matcher` that matches all occurrences of an adjective followed by an organization and store the 5 most common ocurrences in a list named `most_common_adj_ents`.

In [21]:
#Build a `Matcher` that matches all ocurrences of an adjective followed by
#and organization and store the 5 most common ocurrences in a list named most_common_adj_ents
#
#matcher = ...
#
#pattern = [..., ...]
#matcher.add(...)
# 
#...
#
#most_common_adj_ents = ...

# YOUR CODE HERE
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "ADJ"}, {"ENT_TYPE": "ORG"}]
matcher.add('adj_orgs', None, pattern)

adj_organizations = []
for doc in docs:
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]  # the matched span
        span_text = span.text  # the span as a string
        adj_organizations.append(span_text)

most_common_adj_ents = Counter(adj_organizations).most_common(5)

In [22]:
most_common_adj_ents

[('only 150p', 5),
 ('top Sony', 4),
 ('Latest Motorola', 3),
 ('wk TXT', 3),
 ('genuine HMV', 2)]

In [23]:
adj_ent_hash = "4dbad11fe79c962e80f0ad3dc3ef788eaa4b7412c6c99f1d1fe78586486e4afe"  # sha256('top Sony')
assert len(most_common_adj_ents) == 5
assert adj_ent_hash in (sha256(adj_ent.encode()).hexdigest() for adj_ent, count in most_common_adj_ents)

### Q2.b) Create Numerical Features

You start thinking what features could actually be useful for solving your problem. One possible factor that may help is to know the number of adjectives and verbs used, the number of entities, and the length of the messages. 

Add extra fields to the `df` dataframe with the count of the number of adjectives and verbs, entities (all types), and the length of the document. For this consider the adjectives, verbs, and entities as those identified by SpaCy and the length of the text as the count of characters. 

*note: the number of verbs and adjectives should be summed in a single variable*

Assign the number of verbs and adjectives to a new column called `n_verb_adjs`, the number of entities to a column called `n_ents`, and the length of the message to a column called `len_message`.

In [24]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [25]:
#Assign the number of verbs and adjectives to a new column called n_verbs_adjs,
#the number of entities to a column called n_ents,
#and the lenght of the message to a column called len_message
#
# Hint: you can iterate over the tokens in Spacy doc to inspect them 
# for doc in all docs:
#    print(doc.ents)
#    for token in doc:
#        print(token.pos_)

# n_ents = []
# n_verb_adjs = []
# len_message = []
#
#for doc in all_docs:
#    count_ents = 0
#    count_verb_adjs = 0
#    ...
#    n_ents.append(count_ents)
#    n_verb_adjs.append(count_verb_adjs)
#    len_message.append(...)
#    
#df['n_verb_adjs'] = n_verb_adjs
#df['n_ents'] = n_ents
#df['len_message'] = len_message

# YOUR CODE HERE
n_ents = []
n_verb_adjs = []
len_message = []

for doc in all_docs:
    ents_count = len(doc.ents)
    len_text = len(doc.text)
    
    verb_adjs_count = 0
    for token in doc:
        if token.pos_ == "VERB":
            verb_adjs_count += 1
        elif token.pos_ == "ADJ":
            verb_adjs_count += 1
    
    n_ents.append(ents_count)
    len_message.append(len_text)   
    n_verb_adjs.append(verb_adjs_count)
     

df['n_ents'] = n_ents
df['len_message'] = len_message
df['n_verb_adjs'] = n_verb_adjs


# for doc in docs:
#     for ent in doc.ents:
#         if ent.label_ == "ORG":
#             organizations.append(ent.text)
#             #print(ent.text, ent.start_char, ent.end_char, ent.label_)    


In [26]:
assert all(col in df.columns for col in ('n_verb_adjs', 'n_ents', 'len_message'))
assert np.allclose(df.n_verb_adjs.sum(), 23979, 20)
assert np.allclose(df.n_ents.sum(), 5537, 20)

## Q3 - Pipelines and Feature Unions
It is now time for you to leverage on your newly built features and construct pipelines that can be fed to a classifier. You decide to use a [Random Forest Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) as you hear from industry experts it tends to work well for text classification problems.

In [27]:
# split data into train and test sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [28]:
class Selector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a column from the dataframe to perform additional transformations on
    """ 
    def __init__(self, key):
        self.key = key
        
    def fit(self, X, y=None):
        return self
    

class TextSelector(Selector):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def transform(self, X):
        return X[self.key]
    
    
class NumberSelector(Selector):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def transform(self, X):
        return X[[self.key]]

    
def get_accuracy(feats, train_data, test_data):
    """
    Return the accuracy on the test_data by using a RandomForestClassifier trained on the 
    train_data with the features described by feats
    """

    pipeline = Pipeline([
        ('features',feats),
        ('classifier', RandomForestClassifier(random_state = 42, n_estimators=10)),
    ])

    pipeline.fit(train_data, train_data.label)

    preds = pipeline.predict(test_data)
    accuracy = np.mean(preds == test_data.label)
    
    print("Accuracy: {:.4f}".format(accuracy))
    
    return accuracy

In [29]:
train_data.head()

,label,message,n_ents,len_message,n_verb_adjs
1978,ham,No I'm in the same boat. Still here at my moms...,1,83,3
3989,spam,(Bank of Granite issues Strong-Buy) EXPLOSIVE ...,2,137,2
3935,ham,They r giving a second chance to rahul dengra.,2,46,3
4078,ham,O i played smash bros &lt;#&gt; religiously.,1,46,1
4086,spam,PRIVATE! Your 2003 Account Statement for 07973...,7,148,3


### Q3.a) Build a Feature Union
You hypothesize that combining the text and numerical features could help you build a strong classifier. 

Use `FeatureUnion` to join the text features extracted from a standard sklearn `TfidfVectorizer` (with $ngram\_range=(1,2)$) and the numeric feature of the length of the messages scaled to zero mean and unit variance *[hint](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)*. Assign them to a variable named `feats`.

In [30]:
#text_pipe = Pipeline(...)
#len_pipe =  Pipeline(...)
#feats = FeatureUnion(...)

# YOUR CODE HERE
text_pipe = Pipeline([('selector', TextSelector("message")),
                      ('tfidf', TfidfVectorizer(ngram_range=(1,2)))
                     ])
len_pipe =  Pipeline([('selector', NumberSelector("len_message")),
                      ('standard', StandardScaler())
                     ])
feats = FeatureUnion([('text', text_pipe), 
                      ('length', len_pipe)
                     ])

In [31]:
assert isinstance(feats, FeatureUnion)
assert any(isinstance(obj, Selector) for obj in feats.transformer_list[0][1])
assert any(isinstance(obj, TfidfVectorizer) for obj in feats.transformer_list[0][1])
assert np.allclose(get_accuracy(feats, train_data, test_data), 0.9668, 0.01)

TypeError: 'Pipeline' object is not iterable

### Q3.b) Add more features
You decide to try adding the number of verbs and adjectives to your features to see if they can improve the performance of your classifier. 

Add the number of verbs and adjs `n_verb_adjs` that you computed in Q2.b to your features. Assign your features to `feats_v2`.

In [32]:
#verbs = Pipeline(...)
#...
#feats_v2 = FeatureUnion(...)

# YOUR CODE HERE
verbs = Pipeline([('selector', NumberSelector("n_verb_adjs")), 
                  ('standard', StandardScaler())
                 ])
                  
feats_v2 = FeatureUnion([('text', text_pipe), 
                         ('length', len_pipe),
                         ('verb_adj', verbs)
                        ])

In [33]:
accuracy = get_accuracy(feats_v2, train_data, test_data)
assert np.allclose(accuracy, 0.9704, 0.01)

C:\Users\jnpicao\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\jnpicao\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\jnpicao\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\jnpicao\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\jnpicao\AppData\Local\Continuum\anaco

Accuracy: 0.9668


### Q3.c) Add the entities feature
You try to improve your model even further by including the number of entities `n_ents` feature that you created in Q2.b above. 

Add the number of entities to your features and assign the result to `feats_v3` (**no need to scale** the features this time).

In [34]:
#...
#feats_v3 = FeatureUnion(...)

# YOUR CODE HERE

n_ents = Pipeline([('selector', NumberSelector("n_ents")), 
                   ('standard', StandardScaler())
                  ])

feats_v3 = FeatureUnion([('text', text_pipe), 
                         ('length', len_pipe),
                         ('verb_adj', verbs),
                         ('ents', n_ents)
                        ])

In [35]:
accuracy = get_accuracy(feats_v3, train_data, test_data)
assert np.allclose(accuracy, 0.9659, 0.01)

C:\Users\jnpicao\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\jnpicao\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\jnpicao\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\jnpicao\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\jnpicao\AppData\Local\Continuum\anaco

Accuracy: 0.9596


You realize that your accuracy actually decreased, which reminds you that more features does not necessarily mean better results.

## Conclusion

You realize you can get fairly high accuracy on the spam problem using a fairly simple solution. You know there are many things you could improve and many further paths you could choose in order to try to take your classifier to the next level, but you decide to leave that challenge for another day. 